In [1]:
import sys
#sys.path
sys.executable

'C:\\Users\\AlexChen\\AppData\\Local\\Microsoft\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\python.exe'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import torchvision
#from d2l import torch as d2l

In [3]:
# step 1. Data prepartation
test_data = pd.read_csv("california-house-prices/test.csv")
train_data = pd.read_csv("california-house-prices/train.csv")
###print(train_data.shape)
###print(train_data.iloc[0:4],[0,1,-1,-2])

In [4]:
# Step2. Data Preprocess
all_features = pd.concat((train_data.iloc[:,1:],train_data.iloc[:,1:]))
numeric_features = all_features.dtypes[all_features.dtypes != "object"].index
all_features[numeric_features] = all_features[numeric_features].apply(lambda x:(x-x.mean())/(x.std()))
all_features[numeric_features] = all_features[numeric_features].fillna(0)
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape

(94878, 129676)

In [6]:
n_train = train_data.shape[0]
# dtype=torch.float32
train_features = torch.tensor(all_features[:n_train].values, dtype=float32)
test_features = torch.tensor(all_features[n_train:].values, dtype=float32)
train_labels = torch.tensor(train_data.SalePrice.values.reshape(-1,1), dtype=float32)

MemoryError: Unable to allocate 45.8 GiB for an array with shape (129676, 47439) and data type object

In [ ]:
'''
class model(nn.Module)：
    def __init__(self, input_size):
        super(model, self).__init__()
        self.fc ==nn.Linear(input_size, 1)
        # Optimizer
        self.optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay
    
    def log_rmse(net, features, labels):
        clipped_preds = torch.clamp(net(features), 1, float("inf"))
        rmse = torch.sqrt(loss(torch.log(clipped_preds),torch.log(labels)))
        return rmse.item()

    # Train Model
    def train(net, train_features, train_labels, test_features, test_labels,
              num_epochs, learning_rate, weigth_decay, batch_size):
        train_ls, test_ls = [], []
        train_iter = d2l.load_array((train_features, train_labels), batch_size)
'''

In [7]:
# Step 3. Desgin the basic model
# Loss func
def log_rmse(net, features, labels):
    clipped_preds = torch.clamp(net(features), 1, float("inf"))
    rmse = torch.sqrt(loss(torch.log(clipped_preds),torch.log(labels)))
    return rmse.item()

# Train Model
def train(net,train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weigth_decay, batch_size, ):
    train_ls, test_ls = [], []
    train_iter = d2l.load_array((train_features, train_labels), batch_size)
    #criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay)
    for epoch in range(num_epochs):
        for X, y in train_iter:
            optimizer.zero_grad()
            l = loss(net(X),y)
            l.backward()
            optimizer.step()
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    return train_ls, test_ls

# k_fold_validated
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in rang(k):
        idx = slice(j * fold_size, (j+1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid

def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net()
        train_ls, valid_ls = train(net, optimizer, *data, num_epochs, learning_rate, weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        if i==0:
            d2l.plot(list(range(1,num_epochs+1)), [train_ls, valid_ls], 
                     xlabel='epoch', ylabel='rsm', xlim=[1,num_epochs+1], legend=['train','valid'], yscale='log')
        print(f'fold{i+1}, train log rmse{float(train_ls[-1]):f}',
             f'valid log rmse{float(valid_ls[-1]):f}')
    return train_l_sum / k, valid_l_sum / k

    

In [9]:
k, num_epochs, lr, weight_decay, batch_size = 5, 100, 5, 0, 64
train_l ,valid_l = k_fold(k, train_features, train_labels, nump_epochs, lr,
                          weight_decay, batch_size)

NameError: name 'train_features' is not defined